# Data Processing

## What is in this file ? (Not now, TODO while needed)

This file contains the code to process the data. 
We will first to load the two datasets, then we will merge them. We will create new feature with One Hot Encoding and wipe previous encoding.

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import networkx as nx
import numpy as np

## Loading the data

In [2]:
file1 = 'data/globalterrorismdb_2021Jan-June_1222dist.xlsx'
file2 = 'data/globalterrorismdb_0522dist.xlsx'

df1 = pd.read_excel(file1)
df2 = pd.read_excel(file2)

df1 = pd.concat([df1, df2], axis=0)
df1.shape

(214666, 135)

## Cleaning the dataset


In [3]:
# Colunms to be dropped
## alternative, region, attacktype1, attacktype2, attacktype3, weaptype1, weapsubtype1, weapsubtype3, weaptype4, weapsubtype4, targtype1, 
## targsubtype1, natlty1, targtype2, targsubtype2, natlty2, targtype3,  targsubtype3, natlty3, 
## propextent, claimmode, hostkidoutcome, country

pd.set_option('display.max_columns', None)
df1.drop(columns=['alternative',
                  'region', 
                  'weapsubtype1', 
                  'weapsubtype3', 
                  'weapsubtype4', 
                  'targsubtype1', 
                  'natlty1', 
                  'targsubtype2', 
                  'natlty2', 
                  'targsubtype3', 
                  'natlty3',
                  'propextent',
                  'hostkidoutcome',
                  'country'
                  ], 
                  inplace=True, errors='ignore')
print(df1.shape)
df1.head()

(214666, 121)


,eventid,iyear,imonth,iday,approxdate,extended,resolution,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1_txt,corp1,target1,natlty1_txt,targtype2,targtype2_txt,targsubtype2_txt,corp2,target2,natlty2_txt,targtype3,targtype3_txt,targsubtype3_txt,corp3,target3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1,weaptype1_txt,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,202101010004,2021,1,1,NaN,0,NaT,Egypt,Middle East & North Africa,North Sinai,Beir al-Abd,31.016779,33.008587,1.0,0,NaN,01/01/2021: An explosive device detonated targ...,1,1,0,1,Insurgency/Guerilla Action,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,4,Military,Military Transportation/Vehicle (excluding con...,Egyptian Armed Forces,Vehicle,Egypt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,Remote Trigger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A remote-controlled roadside bomb was used in ...,2.0,0.0,0.0,5.0,0.0,0.0,1,Unknown,-99.0,Vehicle damaged,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Egyptian officials: Roadside bombing in Sinai...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN
1,202101010005,2021,1,1,01/01/2021,0,NaT,India,South Asia,Jammu and Kashmir,Pampore,34.010230,74.917261,1.0,0,NaN,01/01/2021: Assailants threw a grenade at a Ce...,1,1,1,0,NaN,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,3,Police,"Police Building (headquarters, station, school)",Central Reserve Police Force (CRPF),Camp,India,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,Grenade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Terror module busted, seven arrested in J&K,""...","""Militants throw grenade on CRPF,"" HT Syndicat...",NaN,START Primary Collection,-9,-9,0,-9,NaN
2,202101010006,2021,1,1,NaN,0,NaT,Yemen,Middle East & North Africa,Al Hudaydah,Hudaydah,14.782391,42.973093,1.0,1,NaN,01/01/2021: Assailants launched an artillery s...,1,1,1,1,State Actors,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,14,Private Citizens & Property,"Procession/Gathering (funeral, wedding, birthd...",Almansour Wedding Hall,Wedding Hall,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,"Projectile (rockets, mortars, RPGs, etc.)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A mortar was used in the attack.,5.0,0.0,0.0,7.0,0.0,0.0,1,Minor (likely < $1 million),-99.0,Building damaged,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,There is doubt that this incident meets terror...,"""Civilian reportedly dies of injuries from Hou...","""UN troubled by civilian deaths in Yemen's Hud...","""Five women killed in Yemen wedding attack,"" G...",START Primary Collection,0,0,0,0,NaN
3,202101010009,2021,1,1,NaN,0,NaT,Afghanistan,South Asia,Ghor,Feroz Koh,34.521315,65.24

In [4]:
# replace all the 0 in iday value with 1
df1['iday'] = df1['iday'].replace(0, 1)
df1['imonth'] = df1['imonth'].replace(0, 1)

In [5]:
# DATE PREPROCESSING

df1['begin_date'] = pd.to_datetime(df1['iyear'].astype(str) + '-' + df1['imonth'].astype(str) + '-' + df1['iday'].astype(str), errors='raise', format='%Y-%m-%d')
df1.drop(columns=['iyear', 'imonth', 'iday'], inplace=True)
df1.shape

(214666, 119)

In [6]:
df1['begin_date']

0        2021-01-01
1        2021-01-01
2        2021-01-01
3        2021-01-01
4        2021-01-02
            ...    
209701   2020-12-31
209702   2020-12-31
209703   2020-12-31
209704   2020-12-31
209705   2020-12-31
Name: begin_date, Length: 214666, dtype: datetime64[ns]

In [7]:
cols = df1.columns.tolist()
cols.insert(1, cols.pop(cols.index('begin_date')))
df1 = df1[cols]
df1.head()

,eventid,begin_date,approxdate,extended,resolution,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1_txt,corp1,target1,natlty1_txt,targtype2,targtype2_txt,targsubtype2_txt,corp2,target2,natlty2_txt,targtype3,targtype3_txt,targsubtype3_txt,corp3,target3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1,weaptype1_txt,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,202101010004,2021-01-01,NaN,0,NaT,Egypt,Middle East & North Africa,North Sinai,Beir al-Abd,31.016779,33.008587,1.0,0,NaN,01/01/2021: An explosive device detonated targ...,1,1,0,1,Insurgency/Guerilla Action,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,4,Military,Military Transportation/Vehicle (excluding con...,Egyptian Armed Forces,Vehicle,Egypt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,Remote Trigger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A remote-controlled roadside bomb was used in ...,2.0,0.0,0.0,5.0,0.0,0.0,1,Unknown,-99.0,Vehicle damaged,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Egyptian officials: Roadside bombing in Sinai...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN
1,202101010005,2021-01-01,01/01/2021,0,NaT,India,South Asia,Jammu and Kashmir,Pampore,34.010230,74.917261,1.0,0,NaN,01/01/2021: Assailants threw a grenade at a Ce...,1,1,1,0,NaN,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,3,Police,"Police Building (headquarters, station, school)",Central Reserve Police Force (CRPF),Camp,India,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,Grenade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Terror module busted, seven arrested in J&K,""...","""Militants throw grenade on CRPF,"" HT Syndicat...",NaN,START Primary Collection,-9,-9,0,-9,NaN
2,202101010006,2021-01-01,NaN,0,NaT,Yemen,Middle East & North Africa,Al Hudaydah,Hudaydah,14.782391,42.973093,1.0,1,NaN,01/01/2021: Assailants launched an artillery s...,1,1,1,1,State Actors,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,14,Private Citizens & Property,"Procession/Gathering (funeral, wedding, birthd...",Almansour Wedding Hall,Wedding Hall,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,"Projectile (rockets, mortars, RPGs, etc.)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A mortar was used in the attack.,5.0,0.0,0.0,7.0,0.0,0.0,1,Minor (likely < $1 million),-99.0,Building damaged,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,There is doubt that this incident meets terror...,"""Civilian reportedly dies of injuries from Hou...","""UN troubled by civilian deaths in Yemen's Hud...","""Five women killed in Yemen wedding attack,"" G...",START Primary Collection,0,0,0,0,NaN
3,202101010009,2021-01-01,NaN,0,NaT,Afghanistan,South Asia,Ghor,Feroz Koh,34.521315,65.2

In [8]:
df1[df1['resolution'].notnull()]

df1['end_date'] = pd.to_datetime(df1['resolution'], errors='coerce', format='%Y-%m-%d')


In [9]:
df1.loc[df1['resolution'].isna(), 'end_date'] = df1.loc[df1['resolution'].isnull(), 'begin_date']


In [10]:
cols = df1.columns.tolist()
cols.insert(2, cols.pop(cols.index('end_date')))
df1 = df1[cols]
df1.head()

,eventid,begin_date,end_date,approxdate,extended,resolution,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1_txt,corp1,target1,natlty1_txt,targtype2,targtype2_txt,targsubtype2_txt,corp2,target2,natlty2_txt,targtype3,targtype3_txt,targsubtype3_txt,corp3,target3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1,weaptype1_txt,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,202101010004,2021-01-01,2021-01-01,NaN,0,NaT,Egypt,Middle East & North Africa,North Sinai,Beir al-Abd,31.016779,33.008587,1.0,0,NaN,01/01/2021: An explosive device detonated targ...,1,1,0,1,Insurgency/Guerilla Action,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,4,Military,Military Transportation/Vehicle (excluding con...,Egyptian Armed Forces,Vehicle,Egypt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,Remote Trigger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A remote-controlled roadside bomb was used in ...,2.0,0.0,0.0,5.0,0.0,0.0,1,Unknown,-99.0,Vehicle damaged,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Egyptian officials: Roadside bombing in Sinai...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN
1,202101010005,2021-01-01,2021-01-01,01/01/2021,0,NaT,India,South Asia,Jammu and Kashmir,Pampore,34.010230,74.917261,1.0,0,NaN,01/01/2021: Assailants threw a grenade at a Ce...,1,1,1,0,NaN,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,3,Police,"Police Building (headquarters, station, school)",Central Reserve Police Force (CRPF),Camp,India,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,Grenade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Terror module busted, seven arrested in J&K,""...","""Militants throw grenade on CRPF,"" HT Syndicat...",NaN,START Primary Collection,-9,-9,0,-9,NaN
2,202101010006,2021-01-01,2021-01-01,NaN,0,NaT,Yemen,Middle East & North Africa,Al Hudaydah,Hudaydah,14.782391,42.973093,1.0,1,NaN,01/01/2021: Assailants launched an artillery s...,1,1,1,1,State Actors,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,14,Private Citizens & Property,"Procession/Gathering (funeral, wedding, birthd...",Almansour Wedding Hall,Wedding Hall,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,"Projectile (rockets, mortars, RPGs, etc.)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A mortar was used in the attack.,5.0,0.0,0.0,7.0,0.0,0.0,1,Minor (likely < $1 million),-99.0,Building damaged,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,There is doubt that this incident meets terror...,"""Civilian reportedly dies of injuries from Hou...","""UN troubled by civilian deaths in Yemen's Hud...","""Five women killed in Yemen wedding attack,"" G...",START Primary Collection,0,0,0,0,NaN
3,202101010009,2021-01-01,2021-01-01,NaN,0,NaT

In [11]:
df1.drop(columns=['resolution', 'approxdate', 'extended'], inplace=True)

In [12]:
df1.loc[df1['property'] == 0, 'propextent_txt'] = 'None'

In [13]:
df1.loc[df1["property"] == 0, 'propvalue'] = 0
df1.loc[df1["propvalue"] == -99, 'propvalue'] =  pd.NA

In [14]:
df1[df1["propvalue"] == -99]

,eventid,begin_date,end_date,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1_txt,corp1,target1,natlty1_txt,targtype2,targtype2_txt,targsubtype2_txt,corp2,target2,natlty2_txt,targtype3,targtype3_txt,targsubtype3_txt,corp3,target3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1,weaptype1_txt,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related


In [15]:
df1.drop(columns=['property'], inplace=True)

In [16]:
df1.drop(columns=['nkillus', 'nwoundus', 'nhostkidus'], inplace=True)

In [17]:
df1.head()

,eventid,begin_date,end_date,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1_txt,corp1,target1,natlty1_txt,targtype2,targtype2_txt,targsubtype2_txt,corp2,target2,natlty2_txt,targtype3,targtype3_txt,targsubtype3_txt,corp3,target3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1,weaptype1_txt,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4_txt,weapdetail,nkill,nkillter,nwound,nwoundte,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,202101010004,2021-01-01,2021-01-01,Egypt,Middle East & North Africa,North Sinai,Beir al-Abd,31.016779,33.008587,1.0,0,NaN,01/01/2021: An explosive device detonated targ...,1,1,0,1,Insurgency/Guerilla Action,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,4,Military,Military Transportation/Vehicle (excluding con...,Egyptian Armed Forces,Vehicle,Egypt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,Remote Trigger,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A remote-controlled roadside bomb was used in ...,2.0,0.0,5.0,0.0,Unknown,NaN,Vehicle damaged,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Egyptian officials: Roadside bombing in Sinai...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN
1,202101010005,2021-01-01,2021-01-01,India,South Asia,Jammu and Kashmir,Pampore,34.010230,74.917261,1.0,0,NaN,01/01/2021: Assailants threw a grenade at a Ce...,1,1,1,0,NaN,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,3,Police,"Police Building (headquarters, station, school)",Central Reserve Police Force (CRPF),Camp,India,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,Grenade,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,None,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Terror module busted, seven arrested in J&K,""...","""Militants throw grenade on CRPF,"" HT Syndicat...",NaN,START Primary Collection,-9,-9,0,-9,NaN
2,202101010006,2021-01-01,2021-01-01,Yemen,Middle East & North Africa,Al Hudaydah,Hudaydah,14.782391,42.973093,1.0,1,NaN,01/01/2021: Assailants launched an artillery s...,1,1,1,1,State Actors,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,14,Private Citizens & Property,"Procession/Gathering (funeral, wedding, birthd...",Almansour Wedding Hall,Wedding Hall,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,"Projectile (rockets, mortars, RPGs, etc.)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A mortar was used in the attack.,5.0,0.0,7.0,0.0,Minor (likely < $1 million),NaN,Building damaged,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,There is doubt that this incident meets terror...,"""Civilian reportedly dies of injuries from Hou...","""UN troubled by civilian deaths in Yemen's Hud...","""Five women killed in Yemen wedding attack,"" G...",START Primary Collection,0,0,0,0,NaN
3,202101010009,2021-01-01,2021-01-01,Afghanistan,South Asia,Ghor,Feroz Koh,34.521315,65.249242,1.0,1,NaN,01/01/2021: Assailants opened fire on the vehi...,1,1,1,1,State Actors,0.0,1,0,1,Assassinat

In [18]:
weapon__type_dict = {1: 'Biological', 2: 'Chemical', 3: 'Radiological', 4: 'Nuclear', 5: 'Firearms', 6: 'Explosives', 7: 'Fake Weapons', 8: 'Incendiary', 9: 'Melee', 10: 'Vehicle', 11: 'Sabotage Equipment', 12: 'Other', 13: 'Unknown'}

for key, value in weapon__type_dict.items():
    df1["weapon_"+value.lower()] = 0

# for each create column fill with 1 if the value is present
for key, value in weapon__type_dict.items():
    df1.loc[df1['weaptype1'] == key, "weapon_"+value.lower()] = 1
    df1.loc[df1['weaptype2'] == key, "weapon_"+value.lower()] = 1
    df1.loc[df1['weaptype3'] == key, "weapon_"+value.lower()] = 1
    df1.loc[df1['weaptype4'] == key, "weapon_"+value.lower()] = 1


df1.drop(columns=['weaptype1','weaptype1_txt', 'weaptype2', 'weaptype2_txt','weaptype3','weaptype3_txt', 'weaptype4','weaptype4_txt'], inplace=True)

df1.head()

,eventid,begin_date,end_date,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1_txt,corp1,target1,natlty1_txt,targtype2,targtype2_txt,targsubtype2_txt,corp2,target2,natlty2_txt,targtype3,targtype3_txt,targsubtype3_txt,corp3,target3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weapsubtype1_txt,weapsubtype2,weapsubtype2_txt,weapsubtype3_txt,weapsubtype4_txt,weapdetail,nkill,nkillter,nwound,nwoundte,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,weapon_biological,weapon_chemical,weapon_radiological,weapon_nuclear,weapon_firearms,weapon_explosives,weapon_fake weapons,weapon_incendiary,weapon_melee,weapon_vehicle,weapon_sabotage equipment,weapon_other,weapon_unknown
0,202101010004,2021-01-01,2021-01-01,Egypt,Middle East & North Africa,North Sinai,Beir al-Abd,31.016779,33.008587,1.0,0,NaN,01/01/2021: An explosive device detonated targ...,1,1,0,1,Insurgency/Guerilla Action,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,4,Military,Military Transportation/Vehicle (excluding con...,Egyptian Armed Forces,Vehicle,Egypt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Remote Trigger,NaN,NaN,NaN,NaN,A remote-controlled roadside bomb was used in ...,2.0,0.0,5.0,0.0,Unknown,NaN,Vehicle damaged,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Egyptian officials: Roadside bombing in Sinai...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0
1,202101010005,2021-01-01,2021-01-01,India,South Asia,Jammu and Kashmir,Pampore,34.010230,74.917261,1.0,0,NaN,01/01/2021: Assailants threw a grenade at a Ce...,1,1,1,0,NaN,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,3,Police,"Police Building (headquarters, station, school)",Central Reserve Police Force (CRPF),Camp,India,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Grenade,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,None,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Terror module busted, seven arrested in J&K,""...","""Militants throw grenade on CRPF,"" HT Syndicat...",NaN,START Primary Collection,-9,-9,0,-9,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0
2,202101010006,2021-01-01,2021-01-01,Yemen,Middle East & North Africa,Al Hudaydah,Hudaydah,14.782391,42.973093,1.0,1,NaN,01/01/2021: Assailants launched an artillery s...,1,1,1,1,State Actors,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,14,Private Citizens & Property,"Procession/Gathering (funeral, wedding, birthd...",Almansour Wedding Hall,Wedding Hall,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Projectile (rockets, mortars, RPGs, etc.)",NaN,NaN,NaN,NaN,A mortar was used in the attack.,5.0,0.0,7.0,0.0,Minor (likely < $1 million),NaN,Building damaged,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,There is doubt that this incident meets terror...,"""Civilian reportedly dies of injuries from Hou...","""UN troubled by civilian deaths in Yemen's Hud...","""Five women killed in Yemen wedding attack,"" G...",START Primary Collection,0,0,0,0,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0
3,202101010009,2021-01-01,2021-01-01,Afghanistan,South Asia,Ghor,Feroz Koh,34.521315,65.249242,1.0,1,N

In [19]:
df1['weapon_unknown'].sum()

np.int64(22348)

In [20]:
df1.drop(columns=['location',  'addnotes','weapdetail', 'scite1', 'scite2', 'scite3', 'motive' ], inplace=True, errors='ignore')

In [21]:
## ATTACK TYPE

attack_type_dict = {1: 'Assassination', 2: 'Armed Assault', 3: 'Bombing/Explosion', 4: 'Hijacking', 5: 'Hostage Taking (Barricade Incident)', 6: 'Hostage Taking (Kidnapping)', 7: 'Facility/Infrastructure Attack', 8: 'Unarmed Assault', 9: 'Unknown'}

for key, value in attack_type_dict.items():
    df1["attack_"+value.lower()] = 0

for key, value in attack_type_dict.items():
    df1.loc[df1['attacktype1'] == key, "attack_"+value.lower()] = 1
    df1.loc[df1['attacktype2'] == key, "attack_"+value.lower()] = 1
    df1.loc[df1['attacktype3'] == key, "attack_"+value.lower()] = 1

df1.drop(columns=['attacktype1', 'attacktype1_txt', 'attacktype2', 'attacktype2_txt', 'attacktype3', 'attacktype3_txt'], inplace=True, errors='ignore')


In [22]:
df1[df1['nhostkid'].notnull()]

,eventid,begin_date,end_date,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,summary,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,targtype1,targtype1_txt,targsubtype1_txt,corp1,target1,natlty1_txt,targtype2,targtype2_txt,targsubtype2_txt,corp2,target2,natlty2_txt,targtype3,targtype3_txt,targsubtype3_txt,corp3,target3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weapsubtype1_txt,weapsubtype2,weapsubtype2_txt,weapsubtype3_txt,weapsubtype4_txt,nkill,nkillter,nwound,nwoundte,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,weapon_biological,weapon_chemical,weapon_radiological,weapon_nuclear,weapon_firearms,weapon_explosives,weapon_fake weapons,weapon_incendiary,weapon_melee,weapon_vehicle,weapon_sabotage equipment,weapon_other,weapon_unknown,attack_assassination,attack_armed assault,attack_bombing/explosion,attack_hijacking,attack_hostage taking (barricade incident),attack_hostage taking (kidnapping),attack_facility/infrastructure attack,attack_unarmed assault,attack_unknown
33,202101050001,2021-01-05,2021-01-05,Afghanistan,South Asia,Ghazni,Ghazni,33.542623,68.415330,1.0,0,01/05/2021: An assailant poisoned and then sho...,1,1,0,1,Insurgency/Guerilla Action,0.0,1,0,4,Military,"Military Personnel (soldiers, troops, officers...",Afghan National Army (ANA),Soldiers,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Taliban,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,1.0,0.0,1.0,8.0,Personal claim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown Gun Type,1.0,Poisoning,NaN,NaN,2.0,0.0,5.0,0.0,Unknown,NaN,Weapons and equipment stolen,1.0,1.0,NaN,-99.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Unknown,-99.0,START Primary Collection,0,0,0,0,NaN,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0
47,202101060016,2021-01-06,2021-01-06,United States,North America,District of Columbia,Washington,38.909934,-77.014726,1.0,0,01/06/2021: Between 1:00 and 2:00 PM during a ...,1,1,1,0,NaN,1.0,1,0,2,Government (General),Election-related,United States Congress,2020 Presidential Election Joint Electoral Vot...,United States,2.0,Government (General),Government Building/Facility/Office,Government of the United States,Capitol Building,United States,3.0,Police,Police Security Forces/Officers,Capitol Police,Officers,United States,Oath Keepers,NaN,Proud Boys,NaN,Three Percenters,NaN,0.0,0.0,0.0,0,-99.0,0.0,1.0,5.0,Note left at scene,1.0,5.0,Note left at scene,1.0,5.0,Note left at scene,0.0,Other Explosive Type,3.0,Handgun,Knife or Other Sharp Object,NaN,1.0,1.0,182.0,68.0,Major (likely >= $1 million but < $1 billion),NaN,Capitol building damaged,1.0,-99.0,-99.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Hostage(s) released by perpetrators,-99.0,START Primary Collection,0,0,0,0,"202101060016, 202101060017, 202101060018",0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0
54,202101060030,2021-01-06,2021-01-06,Yemen,Middle East & North Africa,Amanat Al Asimah,Sanaa,15.325444,44.211895,1.0,0,01/06/2021: Assailants abducted an employee of...,1,1,1,0,NaN,1.0,1,0,1,Business,Gas/Oil/Electric,Safer Oil Company,Residence of Engineer: Mohammed Fuad,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,None,0.0,NaN,1.0,1.0,NaN,-99.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Unknown,-99.0,START Primary Collection,0,0,0,0,"202101060029, 202101060030",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
62,202101070032,2021-01-07,2021-01-07,Afghanistan,South Asia,Kunduz,Kunduz,36.705740,68.876969,1.0,0,01/07/2021: Assailants attacked a security pos...,1,1,0,1,Insurgency

In [23]:
df1.loc[df1['ishostkid'] == 0, 'nhostkid'] = 0

df1[df1['nhostkid'] != 0]

df1.loc[df1['nhostkid'] == -99, 'nhostkid'] = pd.NA


In [24]:
target_type_dict = { 1: 'Business', 2:'Government (General)', 3: 'Police', 4: 'Military', 5: 'Abortion Related', 6: 'Airports & Aircraft', 7: 'Government (Diplomatic)', 8: 'Educational Institution', 9: 'Food or Water Supply', 10: 'Journalists & Media', 11: 'Maritime', 12: 'NGO', 13: 'Other', 14: 'Private Citizens & Property', 15: 'Religious Figures/Institutions', 16: 'Telecommunication',}

for key, value in target_type_dict.items():
    df1["target_"+value.lower()] = 0

for key, value in target_type_dict.items():
    df1.loc[df1['targtype1'] == key, "target_"+value.lower()] = 1
    df1.loc[df1['targtype2'] == key, "target_"+value.lower()] = 1
    df1.loc[df1['targtype3'] == key, "target_"+value.lower()] = 1

In [25]:
df1.drop(columns=['targtype1', 'targtype1_txt', 'targtype2', 'targtype2_txt', 'targtype3', 'targtype3_txt', 'target1', 'target2', 'target3'], inplace=True, errors='ignore')

df1.head()

,eventid,begin_date,end_date,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,summary,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,targsubtype1_txt,corp1,natlty1_txt,targsubtype2_txt,corp2,natlty2_txt,targsubtype3_txt,corp3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weapsubtype1_txt,weapsubtype2,weapsubtype2_txt,weapsubtype3_txt,weapsubtype4_txt,nkill,nkillter,nwound,nwoundte,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,weapon_biological,weapon_chemical,weapon_radiological,weapon_nuclear,weapon_firearms,weapon_explosives,weapon_fake weapons,weapon_incendiary,weapon_melee,weapon_vehicle,weapon_sabotage equipment,weapon_other,weapon_unknown,attack_assassination,attack_armed assault,attack_bombing/explosion,attack_hijacking,attack_hostage taking (barricade incident),attack_hostage taking (kidnapping),attack_facility/infrastructure attack,attack_unarmed assault,attack_unknown,target_business,target_government (general),target_police,target_military,target_abortion related,target_airports & aircraft,target_government (diplomatic),target_educational institution,target_food or water supply,target_journalists & media,target_maritime,target_ngo,target_other,target_private citizens & property,target_religious figures/institutions,target_telecommunication
0,202101010004,2021-01-01,2021-01-01,Egypt,Middle East & North Africa,North Sinai,Beir al-Abd,31.016779,33.008587,1.0,0,01/01/2021: An explosive device detonated targ...,1,1,0,1,Insurgency/Guerilla Action,0.0,1,0,Military Transportation/Vehicle (excluding con...,Egyptian Armed Forces,Egypt,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Remote Trigger,NaN,NaN,NaN,NaN,2.0,0.0,5.0,0.0,Unknown,NaN,Vehicle damaged,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,202101010005,2021-01-01,2021-01-01,India,South Asia,Jammu and Kashmir,Pampore,34.010230,74.917261,1.0,0,01/01/2021: Assailants threw a grenade at a Ce...,1,1,1,0,NaN,0.0,1,0,"Police Building (headquarters, station, school)",Central Reserve Police Force (CRPF),India,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Grenade,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,None,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,202101010006,2021-01-01,2021-01-01,Yemen,Middle East & North Africa,Al Hudaydah,Hudaydah,14.782391,42.973093,1.0,1,01/01/2021: Assailants launched an artillery s...,1,1,1,1,State Actors,0.0,1,0,"Procession/Gathering (funeral, wedding, birthd...",Almansour Wedding Hall,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Projectile (rockets, mortars, RPGs, etc.)",NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,Minor (likely < $1 million),NaN,Building damaged,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,0,0,0,0,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,202101010009,2021-01-01,2021-01-01,Afghanistan,South Asia,Ghor,Feroz Koh,34.521315,65.249242,1.0,1,01/01/2021: Assailants opened fire on the vehi...,1,1,1,1,State Actors,0.0,1,0,Radio Journalist/Staff/Facility,Sadai Ghor Radio,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,N

In [26]:
df1[(df1['guncertain1'] != df1['guncertain2']) & df1['guncertain2'].notnull()]

,eventid,begin_date,end_date,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,summary,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,targsubtype1_txt,corp1,natlty1_txt,targsubtype2_txt,corp2,natlty2_txt,targsubtype3_txt,corp3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weapsubtype1_txt,weapsubtype2,weapsubtype2_txt,weapsubtype3_txt,weapsubtype4_txt,nkill,nkillter,nwound,nwoundte,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,weapon_biological,weapon_chemical,weapon_radiological,weapon_nuclear,weapon_firearms,weapon_explosives,weapon_fake weapons,weapon_incendiary,weapon_melee,weapon_vehicle,weapon_sabotage equipment,weapon_other,weapon_unknown,attack_assassination,attack_armed assault,attack_bombing/explosion,attack_hijacking,attack_hostage taking (barricade incident),attack_hostage taking (kidnapping),attack_facility/infrastructure attack,attack_unarmed assault,attack_unknown,target_business,target_government (general),target_police,target_military,target_abortion related,target_airports & aircraft,target_government (diplomatic),target_educational institution,target_food or water supply,target_journalists & media,target_maritime,target_ngo,target_other,target_private citizens & property,target_religious figures/institutions,target_telecommunication
185,202101230017,2021-01-23,2021-01-23,Saudi Arabia,Middle East & North Africa,Riyadh,Riyadh,24.684588,46.702670,1.0,0,01/23/2021: Security forces intercepted and de...,1,1,1,0,NaN,0.0,0,0,Village/City/Town/Suburb,Riyadh City,Saudi Arabia,NaN,NaN,NaN,NaN,NaN,NaN,Alwiya Al-Waad Al-Haq (Truthful Promise Brigades),NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,0.0,1.0,NaN,0,-99.0,0.0,1.0,7.0,"Posted to website, blog, etc.",0.0,NaN,NaN,NaN,NaN,NaN,NaN,Remote Trigger,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,None,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,1,1,0,1,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
214,202101260019,2021-01-26,2021-01-26,Saudi Arabia,Middle East & North Africa,Riyadh,Riyadh,24.684588,46.702670,1.0,0,01/26/2021: Security forces intercepted and de...,1,1,1,0,NaN,0.0,0,0,Village/City/Town/Suburb,Riyadh City,Saudi Arabia,NaN,NaN,NaN,NaN,NaN,NaN,Alwiya Al-Waad Al-Haq (Truthful Promise Brigades),NaN,Kata'ib Hezbollah,NaN,NaN,NaN,0.0,1.0,NaN,0,-99.0,0.0,1.0,7.0,"Posted to website, blog, etc.",0.0,NaN,NaN,NaN,NaN,NaN,NaN,Remote Trigger,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,None,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,1,1,0,1,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
281,202102020016,2021-02-02,2021-02-02,Afghanistan,South Asia,Kabul,Kabul,34.523842,69.140304,1.0,0,02/02/2021: An explosive device attached to th...,1,1,1,0,NaN,0.0,1,0,Labor Union Related,Jamiat-e-Eslah,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,Khorasan Chapter of the Islamic State,NaN,Taliban,NaN,NaN,NaN,0.0,1.0,NaN,0,-99.0,0.0,1.0,7.0,"Posted to website, blog, etc.",0.0,NaN,NaN,NaN,NaN,NaN,NaN,Sticky Bomb,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,Minor (likely < $1 million),NaN,Vehicle damaged,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,0,1,0,1,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
442,202102170014,2021-02-17,2021-02-17,India,South Asia,Jammu and Kashmir,Srinagar,34.083739,74.789903,1.0,0,02/17/2021: Assailants opened fire at the Kris...,1,1,1,0,NaN,0.0,1,0,Restaurant/Bar/Café,Krishna Dhaba,India,NaN,NaN,NaN,NaN,NaN,NaN,The Resistance Front (TRF),NaN,Muslim Janbaz Force,NaN,

In [27]:
df1.loc[df1['alternative_txt'].notnull()]

,eventid,begin_date,end_date,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,summary,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,targsubtype1_txt,corp1,natlty1_txt,targsubtype2_txt,corp2,natlty2_txt,targsubtype3_txt,corp3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weapsubtype1_txt,weapsubtype2,weapsubtype2_txt,weapsubtype3_txt,weapsubtype4_txt,nkill,nkillter,nwound,nwoundte,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,weapon_biological,weapon_chemical,weapon_radiological,weapon_nuclear,weapon_firearms,weapon_explosives,weapon_fake weapons,weapon_incendiary,weapon_melee,weapon_vehicle,weapon_sabotage equipment,weapon_other,weapon_unknown,attack_assassination,attack_armed assault,attack_bombing/explosion,attack_hijacking,attack_hostage taking (barricade incident),attack_hostage taking (kidnapping),attack_facility/infrastructure attack,attack_unarmed assault,attack_unknown,target_business,target_government (general),target_police,target_military,target_abortion related,target_airports & aircraft,target_government (diplomatic),target_educational institution,target_food or water supply,target_journalists & media,target_maritime,target_ngo,target_other,target_private citizens & property,target_religious figures/institutions,target_telecommunication
0,202101010004,2021-01-01,2021-01-01,Egypt,Middle East & North Africa,North Sinai,Beir al-Abd,31.016779,33.008587,1.0,0,01/01/2021: An explosive device detonated targ...,1,1,0,1,Insurgency/Guerilla Action,0.0,1,0,Military Transportation/Vehicle (excluding con...,Egyptian Armed Forces,Egypt,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Remote Trigger,NaN,NaN,NaN,NaN,2.0,0.0,5.0,0.0,Unknown,NaN,Vehicle damaged,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,202101010006,2021-01-01,2021-01-01,Yemen,Middle East & North Africa,Al Hudaydah,Hudaydah,14.782391,42.973093,1.0,1,01/01/2021: Assailants launched an artillery s...,1,1,1,1,State Actors,0.0,1,0,"Procession/Gathering (funeral, wedding, birthd...",Almansour Wedding Hall,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Projectile (rockets, mortars, RPGs, etc.)",NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,Minor (likely < $1 million),NaN,Building damaged,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,0,0,0,0,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,202101010009,2021-01-01,2021-01-01,Afghanistan,South Asia,Ghor,Feroz Koh,34.521315,65.249242,1.0,1,01/01/2021: Assailants opened fire on the vehi...,1,1,1,1,State Actors,0.0,1,0,Radio Journalist/Staff/Facility,Sadai Ghor Radio,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown Gun Type,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,Minor (likely < $1 million),NaN,Vehicle damaged,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
8,202101020010,2021-01-02,2021-01-02,Afghanistan,South Asia,Badghis,Qala-i-Naw,34.990048,63.132541,1.0,0,01/02/2021: Assailants shot and killed a soldi...,1,1,0,1,Insurgency/Guerilla Action,0.0,1,0,"Military Personnel (soldiers, troops, officers...",Afghan National Army (ANA),Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,

In [28]:
df1.loc[df1['related'].notnull(), 'related'] = df1.loc[df1['related'].notnull(), 'related'].apply(lambda x: x.replace('[', '').replace(']', '').split(','))

In [29]:
df1.loc[df1['related'].notnull()]

,eventid,begin_date,end_date,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,summary,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,targsubtype1_txt,corp1,natlty1_txt,targsubtype2_txt,corp2,natlty2_txt,targsubtype3_txt,corp3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weapsubtype1_txt,weapsubtype2,weapsubtype2_txt,weapsubtype3_txt,weapsubtype4_txt,nkill,nkillter,nwound,nwoundte,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,weapon_biological,weapon_chemical,weapon_radiological,weapon_nuclear,weapon_firearms,weapon_explosives,weapon_fake weapons,weapon_incendiary,weapon_melee,weapon_vehicle,weapon_sabotage equipment,weapon_other,weapon_unknown,attack_assassination,attack_armed assault,attack_bombing/explosion,attack_hijacking,attack_hostage taking (barricade incident),attack_hostage taking (kidnapping),attack_facility/infrastructure attack,attack_unarmed assault,attack_unknown,target_business,target_government (general),target_police,target_military,target_abortion related,target_airports & aircraft,target_government (diplomatic),target_educational institution,target_food or water supply,target_journalists & media,target_maritime,target_ngo,target_other,target_private citizens & property,target_religious figures/institutions,target_telecommunication
10,202101030001,2021-01-03,2021-01-03,Somalia,Sub-Saharan Africa,Middle Shebelle,Balcad,2.359127,45.383671,1.0,0,01/03/2021: A roadside explosive device detona...,1,1,1,0,NaN,1.0,1,0,Police Checkpoint,Somali Police Force (SPF),Somalia,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown Explosive Type,NaN,NaN,NaN,NaN,2.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,-9,-9,0,-9,"[202101030001, 202101030002]",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
11,202101030002,2021-01-03,2021-01-03,Somalia,Sub-Saharan Africa,Middle Shebelle,Balcad,2.359127,45.383671,1.0,0,01/03/2021: An explosive device detonated targ...,1,1,0,1,Insurgency/Guerilla Action,1.0,1,0,"Military Personnel (soldiers, troops, officers...",Somali Armed Forces (SAF),Somalia,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown Explosive Type,NaN,NaN,NaN,NaN,1.0,0.0,8.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,-9,-9,0,-9,"[202101030001, 202101030002]",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
31,202101040034,2021-01-04,2021-01-04,Yemen,Middle East & North Africa,Al Hudaydah,Hudaydah,14.782391,42.973093,1.0,0,01/04/2021: Assailants attacked residential ho...,1,1,1,0,NaN,1.0,1,0,House/Apartment/Residence,Not Applicable,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Automatic or Semi-Automatic Rifle,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,None,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,0,0,0,0,"[202101040034, 202101040035]",0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
32,202101040035,2021-01-04,2021-01-04,Yemen,Middle East & North Africa,Al Hudaydah,Bayt al Marui,14.793804,43.074642,1.0,1,01/04/2021: Assailants attacked residential ho...,1,1,1,0,NaN,1.0,1,0,House/Apartment/Residence,Not Applicable,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN

In [30]:
claim_type_dict = {1: 'letter', 2: 'post-incident_call', 3: 'pre-incident_call', 4:'email', 5:'left_note', 6:'video', 7:'social-media-blog',8:'personal_claim',9:'other'}

for key, value in claim_type_dict.items():
    df1["claim_"+value.lower()] = 0

for key, value in claim_type_dict.items():
    df1.loc[df1['claimmode'] == key, "claim_"+value.lower()] = 1
    df1.loc[df1['claimmode2'] == key, "claim_"+value.lower()] = 1
    df1.loc[df1['claimmode3'] == key, "claim_"+value.lower()] = 1

df1.drop(columns=['claimmode', 'claimmode2', 'claimmode3', 'claimed', 'claimmode_txt', 'claimmode2_txt', 'claimmode3_txt'], inplace=True, errors='ignore')

In [31]:
df1.head()

,eventid,begin_date,end_date,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,summary,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,targsubtype1_txt,corp1,natlty1_txt,targsubtype2_txt,corp2,natlty2_txt,targsubtype3_txt,corp3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claim2,claim3,compclaim,weapsubtype1_txt,weapsubtype2,weapsubtype2_txt,weapsubtype3_txt,weapsubtype4_txt,nkill,nkillter,nwound,nwoundte,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,weapon_biological,weapon_chemical,weapon_radiological,weapon_nuclear,weapon_firearms,weapon_explosives,weapon_fake weapons,weapon_incendiary,weapon_melee,weapon_vehicle,weapon_sabotage equipment,weapon_other,weapon_unknown,attack_assassination,attack_armed assault,attack_bombing/explosion,attack_hijacking,attack_hostage taking (barricade incident),attack_hostage taking (kidnapping),attack_facility/infrastructure attack,attack_unarmed assault,attack_unknown,target_business,target_government (general),target_police,target_military,target_abortion related,target_airports & aircraft,target_government (diplomatic),target_educational institution,target_food or water supply,target_journalists & media,target_maritime,target_ngo,target_other,target_private citizens & property,target_religious figures/institutions,target_telecommunication,claim_letter,claim_post-incident_call,claim_pre-incident_call,claim_email,claim_left_note,claim_video,claim_social-media-blog,claim_personal_claim,claim_other
0,202101010004,2021-01-01,2021-01-01,Egypt,Middle East & North Africa,North Sinai,Beir al-Abd,31.016779,33.008587,1.0,0,01/01/2021: An explosive device detonated targ...,1,1,0,1,Insurgency/Guerilla Action,0.0,1,0,Military Transportation/Vehicle (excluding con...,Egyptian Armed Forces,Egypt,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,NaN,NaN,NaN,Remote Trigger,NaN,NaN,NaN,NaN,2.0,0.0,5.0,0.0,Unknown,NaN,Vehicle damaged,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,202101010005,2021-01-01,2021-01-01,India,South Asia,Jammu and Kashmir,Pampore,34.010230,74.917261,1.0,0,01/01/2021: Assailants threw a grenade at a Ce...,1,1,1,0,NaN,0.0,1,0,"Police Building (headquarters, station, school)",Central Reserve Police Force (CRPF),India,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,-99.0,0.0,NaN,NaN,NaN,Grenade,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,None,0.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,202101010006,2021-01-01,2021-01-01,Yemen,Middle East & North Africa,Al Hudaydah,Hudaydah,14.782391,42.973093,1.0,1,01/01/2021: Assailants launched an artillery s...,1,1,1,1,State Actors,0.0,1,0,"Procession/Gathering (funeral, wedding, birthd...",Almansour Wedding Hall,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0,-99.0,0.0,NaN,NaN,NaN,"Projectile (rockets, mortars, RPGs, etc.)",NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,Minor (likely < $1 million),NaN,Building damaged,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,0,0,0,0,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,202101010009,2021-01-01,2021-01-01,Afghanistan,South Asia,Ghor,Feroz Koh,34.521315,65.249242,1.0,1,01/01/2021: Assailants opened fire on the vehi...,1,1,1,1,State Actors,0.0,1,0,Radio Journalist/Staff/Facility,Sadai Ghor Radio,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,

In [32]:
df1.loc[df1.index == 50, 'related']
df1['related'] = df1['related'].astype(object)

In [33]:
# transforme le dataframe en un graph en utilisant networkx
G = nx.Graph()
G.add_nodes_from(df1['eventid'])
# Add node attributes for begin_date and end_date
cols_in_features = ['begin_date', 'end_date', 'latitude', 'longitude', 'crit1', 'crit2', 'crit3', 'targsubtype1_txt', 'gname', 'country_txt', 'success']
cols_in_features.extend([col for col in df1.columns if 'weapon_' in col])
cols_in_features.extend([col for col in df1.columns if 'attack_' in col])
cols_in_features.extend([col for col in df1.columns if 'target_' in col])

for index, row in df1.iterrows():
    for col in cols_in_features:
        if isinstance(row[col], pd.Timestamp):
            G.nodes[row['eventid']][col] = row[col].timestamp()
        else:
            G.nodes[row['eventid']][col] = row[col]


for index, row in df1.iterrows():
    if isinstance(row['related'], list):
        for related_eventid in row['related']:
            G.add_edge(row['eventid'], related_eventid)


In [34]:
df1.to_csv('data/db_preprocessed.csv', index=False)

In [35]:
df1.shape

(214666, 123)

In [36]:
nx.write_graphml(G, 'data/graph_full.graphml')